In [ ]:
import numpy as np
import json
import time
import pickle
import sys
import nltk
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
import unicodedata
from pprint import pprint
import random
import tensorflow as tf
import collections
import time
from math import log10

In [ ]:
"""Convert GloVe file to a dictionary"""
def loadGloveModel(gloveFile):
    print "Loading Glove Model"
    f = open(gloveFile,'r')
    model = dict()
    embedding = []
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        model[word]=np.array([float(val) for val in splitLine[1:]])
    print "Done.",len(model)," words loaded!"
    return model

In [ ]:
%%time
"""Load word as well as character embedding"""
model = loadGloveModel("GloVe/glove.840B.300d.txt")
model_char = loadGloveModel("glove.840B.300d-char.txt")

In [ ]:
"""Reduce a sentence to its word embedding, character embedding, a boolean vector which 
tells if any word is in GloVe dictionary or not and an integer array of start positions of every word"""
def preprocess(sentence):
    wnl = WordNetLemmatizer()
    temp = pos_tag(word_tokenize(sentence))
    y = []
    word_emb = []
    char_emb = []
    word_in_glove = []
    count=0
    word_zeros = np.zeros((300),dtype=float)
    for i,j in temp:
        y.append(count)
        if i==u'``' or i==u"''":
            x='"'
            count+=1
        else:
            if j[0].lower() in ['a','n','v']:
                #lemmatization
                temp_i = wnl.lemmatize(i,j[0].lower())
                # unicode normalization
                x=unicodedata.normalize('NFKD', temp_i).encode('ascii','ignore')
            else:
                temp_i = wnl.lemmatize(i)
                x=unicodedata.normalize('NFKD', temp_i).encode('ascii','ignore')
            count+=len(i)
        while count<len(sentence) and sentence[count]==' ':
            count+=1
        try:
            word_emb.append(model[x])
            word_in_glove.append(1)
        except:
            word_emb.append(word_zeros)
            word_in_glove.append(0)
        temp_char = []
        for k in range(len(x)):
            try:
                temp_char.append(vocab.index(x[k]))
            except:
                pass
        temp_char+=[len(vocab) for _ in range(max_word_len-len(temp_char))]
        char_emb.append(temp_char)
    char_emb=np.array(char_emb)
    word_emb=np.array(word_emb)
    return word_emb, char_emb, word_in_glove, y

In [ ]:
"""Load vocabulary from character embedding GloVe dictionary"""
vocab_emb = []
for k in model_char.keys():
    vocab_emb.append(model_char[k])
vocab_emb.append(np.zeros((300)))
vocab_emb = np.array(vocab_emb)
print(vocab_emb.shape)

In [ ]:
vocab=list(model_char.keys())+[0]

In [ ]:
"""Hyperparameters"""
max_word_len=40
num_gpus=3
word_embedding_dim=300
char_embedding_dim=300
q_words=50
c_words=400
embedding_dim = word_embedding_dim+char_embedding_dim
batch_size=8
batch_size_o=(num_gpus)*batch_size
model_encoder_layers=3
no_of_chars = len(vocab)
hidden_layer_size=96

In [ ]:
"""Initializers and regularizers"""
initializer = lambda: tf.contrib.layers.variance_scaling_initializer(factor=1.0,
                                                             mode='FAN_AVG',
                                                             uniform=True,
                                                             dtype=tf.float32)
initializer_relu = lambda: tf.contrib.layers.variance_scaling_initializer(factor=2.0,
                                                             mode='FAN_IN',
                                                             uniform=False,
                                                             dtype=tf.float32)
regularizer = tf.contrib.layers.l2_regularizer(scale = 3e-7)

In [ ]:
"""Stochastic Depth"""
def layer_dropout(inputs, residual, dropout):
    pred = tf.random_uniform([]) < dropout
    pred = tf.cast(pred,tf.float32)
    ifdrop = tf.nn.dropout(inputs, 1.0 - dropout) + residual
    return pred*residual+(1-pred)*ifdrop

In [ ]:
"""Applies a 2-dimensional convolution over a 3-d or 3-d matrix with bias and activation if specified"""
def conv(inputs, output_size, bias = None, activation = None, kernel_size = 1, name = "conv", reuse=None):
    with tf.variable_scope(name,reuse=reuse):
        def_shape=4
        if len(inputs.shape)==3:
            inputs = tf.expand_dims(inputs,axis=1)
            def_shape=3
        shapes = inputs.shape.as_list()
        filter_shape = [1,kernel_size,shapes[-1],output_size]
        bias_shape = [1,1,1,output_size]
        strides = [1,1,1,1]
        kernel_ = tf.get_variable("kernel_",
                        filter_shape,
                        dtype = tf.float32,
                        regularizer=regularizer,
                        initializer = initializer_relu() if activation is not None else initializer())
        outputs = tf.nn.conv2d(inputs, kernel_, strides, "VALID")
        if bias:
            outputs += tf.get_variable("bias_",
                        bias_shape,
                        regularizer= tf.contrib.layers.l2_regularizer(scale = 3e-7),
                        initializer = tf.zeros_initializer())
        if def_shape==3:
            outputs = tf.squeeze(outputs,axis=1)
        if activation is not None:
            return activation(outputs)
        else:
            return outputs

In [ ]:
"""Depthwise Separable Convolutions"""
def depthconv(x, kernel_size, output_filters, scope_name,reuse=None):
    with tf.variable_scope(scope_name,reuse=reuse):
        shapes = x.shape.as_list()
        depthwise_filter = tf.get_variable("depthwise_filter",
                                        (kernel_size[0], kernel_size[1], shapes[-1], 1),
                                        dtype = tf.float32,
                                        regularizer=regularizer,
                                        initializer = initializer_relu())
        pointwise_filter = tf.get_variable("pointwise_filter",
                                        (1,1,shapes[-1],output_filters),
                                        dtype = tf.float32,
                                        regularizer=regularizer,
                                        initializer = initializer_relu())
        outputs = tf.nn.separable_conv2d(x,
                                        depthwise_filter,
                                        pointwise_filter,
                                        strides = (1,1,1,1),
                                        padding = "SAME")
        b = tf.get_variable("bias",
                outputs.shape[-1],
                regularizer=tf.contrib.layers.l2_regularizer(scale = 3e-7),
                initializer = tf.zeros_initializer())
        outputs += b
        outputs = tf.nn.relu(outputs)
        return outputs

In [ ]:
"""Applies a single head attention layer over V using softmax(QK/d**0.5)V"""
def dot_product_attention(q,k,v,dropout,scope="dot_product_attention"):
    """
    q: a Tensor with shape [batch, heads, length_q, depth_k]
    k: a Tensor with shape [batch, heads, length_kv, depth_k]
    v: a Tensor with shape [batch, heads, length_kv, depth_v]
    """
    with tf.variable_scope(scope):
        # [batch, num_heads, query_length, memory_length]
        logits = tf.matmul(q, k, transpose_b=True)
        logits = logits/(k.shape.as_list()[-1]**0.5)
        weights = tf.nn.softmax(logits, name="attention_weights")
        # dropping out the attention links for each of the heads
        weights = tf.nn.dropout(weights, 1.0 - dropout)
        return tf.matmul(weights, v)

In [ ]:
"""Applies dot attention in parallel with multiplicity equal to the number of heads specified"""
def multihead_attention(queries, units, num_heads, dropout, memory = None, scope = "Multi_Head_Attention",reuse=None):
    with tf.variable_scope(scope,reuse=reuse):
        # Self attention
        if memory is None:
            memory = queries
        memory = conv(memory, 2 * units, name = "memory_projection",reuse=reuse)
        query = conv(queries, units, name = "query_projection",reuse=reuse)
        qshapes = query.shape.as_list()
        Q = tf.reshape(query, [qshapes[0],qshapes[1],num_heads,-1])
        Q = tf.transpose(Q,[0,2,1,3])

        mshapes = memory.shape.as_list()
        M = tf.reshape(memory, [qshapes[0],qshapes[1],num_heads*2,-1])
        M = tf.transpose(M,[0,2,1,3])
        K, V = tf.split(M,2,axis=1)

        x = dot_product_attention(Q,K,V,dropout)
        
        shapes = x.shape.as_list()
        return tf.reshape(tf.transpose(x,[0,2,1,3]),[shapes[0],shapes[2],-1])

In [ ]:
""" Applies num_conv_layers convolutions after layer normalization in num_blocks, then a multihead attention
    and finally, a feed forward layer"""
def encoderblock(x, kernel_size, output_filters, num_conv_layers, scope_name, drop, num_blocks=1, reuse=None):
    with tf.variable_scope(scope_name, reuse=reuse):
        x = tf.expand_dims(x,axis=1)
        x = conv(x,output_filters,name="conv0",reuse=reuse)
        for _ in range(num_blocks):
            with tf.variable_scope("Block"+str(_)):
                for i in range(num_conv_layers):
                    if (i) % 2 == 0:
                        x = tf.nn.dropout(x, 1-drop)
                    if len(x.shape.as_list())==3:
                        x=tf.expand_dims(x,axis=1)
                    y = tf.contrib.layers.layer_norm(x,scope="layer_norm1_%d"%i,reuse=reuse)
                    y = depthconv(y, kernel_size, output_filters, 'dconv'+str(i),reuse=reuse)
                    x = layer_dropout(y,x,drop*float(_*(num_conv_layers+2)+i+1)/float((num_conv_layers + 2) * num_blocks))
                x_res1 = tf.squeeze(x, axis=1)
                x = tf.contrib.layers.layer_norm(x_res1,scope="layer_norm2",reuse=reuse)
                x = tf.nn.dropout(x,1-drop)
                x = multihead_attention(x,output_filters,8,reuse=reuse,dropout=drop)
                x_res2 = layer_dropout(x,x_res1,drop*float(_*(num_conv_layers+2)+num_conv_layers+1)/float((num_conv_layers + 2) * num_blocks))

                x = tf.contrib.layers.layer_norm(x_res2,scope="layer_norm3",reuse=reuse)
                x = tf.nn.dropout(x,1-drop)
                x = conv(x,output_filters,True,activation=tf.nn.relu,name="FFN1",reuse=reuse)
                x = conv(x,output_filters,True,activation=None,name="FFN2",reuse=reuse)
                x = layer_dropout(x,x_res2,drop*float(_*(num_conv_layers+2)+num_conv_layers+2)/float((num_conv_layers + 2) * num_blocks))

        return x

In [ ]:
"""An n-layer Highway network with each layer consisting of 2 convolutions"""
def highway(x, drop, size = None, activation = None, num_layers = 2, scope = "highway",reuse=None):
    with tf.variable_scope(scope,reuse=reuse):
        if size is None:
            size = x.shape.as_list()[-1]
        else:
            x = conv(x, size, name = "input_projection",reuse=reuse)
        for i in range(num_layers):
            T = conv(x, size, bias = True, activation = tf.sigmoid,
                     name = "gate_%d"%i,reuse=reuse)
            H = conv(x, size, bias = True, activation = activation,
                     name = "activation_%d"%i,reuse=reuse)
            H = tf.nn.dropout(H, 1-drop)
            x = H * T + x * (1.0 - T)
        return x

In [ ]:
def average_gradients(tower_grads):
    """Calculate the average gradient for each shared variable across all towers.
    Note that this function provides a synchronization point across all towers.
    Args:
    tower_grads: List of lists of (gradient, variable) tuples. The outer list ranges
        over the devices. The inner list ranges over the different variables.
    Returns:
            List of pairs of (gradient, variable) where the gradient has been averaged
            across all towers.
    """
    average_grads = []
    for grad_and_vars in zip(*tower_grads):

        # Note that each grad_and_vars looks like the following:
        #   ((grad0_gpu0, var0_gpu0), ... , (grad0_gpuN, var0_gpuN))
        grads = [g for g, _ in grad_and_vars]
        grad = tf.reduce_mean(tf.stack(grads), 0)

        # Keep in mind that the Variables are redundant because they are shared
        # across towers. So .. we will just return the first tower's pointer to
        # the Variable.
        v = grad_and_vars[0][1]
        grad_and_var = (grad, v)
        average_grads.append(grad_and_var)
    return average_grads

In [ ]:
"""Starting Learning Rate"""
starter_learning_rate = tf.placeholder(tf.float32,shape=[])
global_step = tf.train.get_or_create_global_step()
# ema = tf.train.ExponentialMovingAverage(decay=0.9999)
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           4000, 1, staircase=True)
# Passing global_step to minimize() will increment it at each step.
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

In [ ]:
"""Placeholders"""
tower_grads = []
losses = []
drop = tf.placeholder_with_default(0.0, shape=())
question_bool = tf.placeholder(tf.float32,[batch_size,q_words],name="question_bool")
question_word = tf.placeholder(tf.float32,[batch_size,q_words,word_embedding_dim],name="question_word")
question_char = tf.placeholder(tf.int32,[batch_size,q_words,None],name="question_char")
context_bool = tf.placeholder(tf.float32,[batch_size,c_words],name="context_bool")
context_word = tf.placeholder(tf.float32,[batch_size,c_words,word_embedding_dim],name="context_word")
context_char = tf.placeholder(tf.int32,[batch_size,c_words,None],name="context_char")
start_ans = tf.placeholder(tf.int32,[batch_size],name="start_ans")
end_ans = tf.placeholder(tf.int32,[batch_size],name="end_ans")

In [ ]:
with tf.variable_scope("Model"):
    final_logits1 = []
    final_logits2 = []
    """Random initialization of some required variables"""
    unk = tf.get_variable("unk",(word_embedding_dim),dtype = tf.float32,
                                            initializer = initializer())
    char_emb = tf.get_variable("char_emb",dtype = tf.float32,
                                            initializer =tf.constant(vocab_emb, dtype=tf.float32))
    zero_word = tf.constant(np.zeros((300)),dtype=tf.float32)
    with tf.variable_scope(tf.get_variable_scope()) as outer_scope:
        unk_tiled = tf.expand_dims(tf.expand_dims(unk,axis=0),axis=0)
        zero_tiled = tf.expand_dims(tf.expand_dims(zero_word,axis=0),axis=0)

        question_bool1 = tf.expand_dims(question_bool,axis=-1)
        context_bool1 = tf.expand_dims(context_bool,axis=-1)
#         print question_bool1
        question_word = (question_bool1*(2-question_bool1)*question_word)+(0.5*(1-question_bool1)*(2-question_bool1)*unk_tiled)+(0.5*question_bool1*(question_bool1-1)*zero_tiled)
        print question_word
        question_word = tf.nn.dropout(question_word,1-drop)

        context_word = (context_bool1*(2-context_bool1)*context_word)+(0.5*(1-context_bool1)*(2-context_bool1)*unk_tiled)+(0.5*context_bool1*(context_bool1-1)*zero_tiled)
        context_word = tf.nn.dropout(context_word,1-drop)


#         print char_emb
        question_char_new = tf.nn.embedding_lookup(char_emb,question_char)
        question_char_new = tf.nn.dropout(question_char_new,1-(drop*0.5))

#         print question_char_new
        context_char_new = tf.nn.embedding_lookup(char_emb,context_char)
        context_char_new = tf.nn.dropout(context_char_new,1-(drop*0.5))

#         print context_char_new
        with tf.variable_scope("Input_Embedding_Layer"):
            """Convolution over char embedding, max pooling and highway network"""
            question_char1 = conv(question_char_new,hidden_layer_size,kernel_size=5,name="q_char_conv0",activation=tf.nn.relu)
            question_char1 = tf.reduce_max(question_char1,axis=2)
            question_emb = tf.concat([question_word,question_char1],axis=-1)
            question_emb = highway(question_emb, drop, scope="highway")
#                     print question_emb

            """Convolution over char embedding, max pooling and highway network"""
            context_char1 = conv(context_char_new,hidden_layer_size,kernel_size=5,name="c_char_conv0",activation=tf.nn.relu)
            context_char1 = tf.reduce_max(context_char1,axis=2)
            context_emb = tf.concat([context_word,context_char1],axis=-1)
            context_emb = highway(context_emb, drop, scope="highway",reuse=True)
#             print context_emb


        with tf.variable_scope("Embedding_encoding_layer"):
            question = encoderblock(question_emb, (1,7), hidden_layer_size,scope_name="encoder_block",num_conv_layers=4,drop=drop)
#             print question

            context = encoderblock(context_emb, (1,7), hidden_layer_size,scope_name="encoder_block",num_conv_layers=4,reuse=True,drop=drop)
#             print context

        with tf.variable_scope("Context-query_attention"):
            """Similarity matrix computation"""
            question = tf.nn.dropout(question,1-drop)
            q = tf.tile(tf.expand_dims(question,axis=1),[1,c_words,1,1])
            context = tf.nn.dropout(context,1-drop)
            c = tf.tile(tf.expand_dims(context,axis=2),[1,1,q_words,1])
            s = conv(tf.concat([q,c,tf.multiply(q,c)],axis=-1),1,name="similarity_matrix")
            s = tf.squeeze(s,axis=-1)
            print s
            s_ = tf.nn.softmax(s,axis=-1)
            a = tf.matmul(s_,question)
#             print a
            b = tf.matmul(s_, tf.matmul(tf.transpose(tf.nn.softmax(s,axis=1),[0,2,1]),context))
#             print b

        with tf.variable_scope("Model_encoder_layer"):
            """3 encoder blocks with shared parameters"""
            enc_input = tf.concat([context,a,context*a,context*b],axis=-1)
#             print enc_input
            enc_input = conv(enc_input,hidden_layer_size,name="conv0")
            enc_input = encoderblock(enc_input, kernel_size=(1,7), output_filters=hidden_layer_size,scope_name="encoder_layer", num_conv_layers=2, num_blocks=5,drop=drop)
#             print enc_input
            output_list = [enc_input]
            for i in range(model_encoder_layers-1):
                temp = encoderblock(output_list[i], kernel_size=(1,7), output_filters=hidden_layer_size,scope_name="encoder_layer", num_conv_layers=2, num_blocks=5,drop=drop,reuse=True)
                print temp
                output_list.append(temp)

        with tf.variable_scope("Output_Layer"):
            """Softmax followed by loss function calculation"""
            start_logits = tf.squeeze(conv(tf.concat([output_list[0], output_list[1]],axis = -1),1, bias = False, name = "start_pointer"),-1)
            end_logits = tf.squeeze(conv(tf.concat([output_list[0], output_list[2]],axis = -1),1, bias = False, name = "end_pointer"), -1)

            logits1 = tf.nn.softmax(start_logits)
            logits2 = tf.nn.softmax(end_logits)
            print logits1
            final_logits1.append(logits1)
            final_logits2.append(logits2)
            start = tf.one_hot(start_ans,c_words)
            end = tf.one_hot(end_ans,c_words)
            print start
            loss1 = tf.nn.softmax_cross_entropy_with_logits_v2(logits=start_logits,labels=start)
            loss2 = tf.nn.softmax_cross_entropy_with_logits_v2(logits=end_logits,labels=end)
            loss = loss1+loss2
            print loss

            grads = optimizer.compute_gradients(loss)
            tower_grads.append(grads)

            losses.append(loss)                



In [ ]:
"""Apply regularization to loss"""
gradients = average_gradients(tower_grads)
avg_loss = tf.reduce_mean(tf.stack(losses))
apply_gradient_op = optimizer.apply_gradients(gradients, global_step)
logits1_f = tf.concat(final_logits1, axis=0)
logits2_f = tf.concat(final_logits2, axis=0)
# params = tf.trainable_variables()
l2_loss = tf.contrib.layers.apply_regularization(tf.contrib.layers.l2_regularizer(scale = 3e-7),tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))
avg_loss+=l2_loss
# with tf.control_dependencies([apply_gradient_op]):
#     # Create the shadow variables, and add ops to maintain moving averages
#     # of var0 and var1. This also creates an op that will update the moving
#     # averages after each training step.  This is what we will use in place
#     # of the usual training op.
#     training_op = ema.apply(params)

In [ ]:
"""Random variables initialization"""
init_op = tf.global_variables_initializer()
config=tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
sess.run(init_op)

In [ ]:
saver = tf.train.Saver()

In [ ]:
saver.restore(sess, "saved_model2/model.ckpt")

In [ ]:
"""F1 score calculation from predicted answer and actual answer in list format"""
def compute_f1(ans_predicted, ans_actual):
    tp = 0
    fp = 0
    tn = 0
    for gold_toks, pred_toks in zip(ans_actual, ans_predicted):
        common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
        num_same = sum(common.values())
        tp+=num_same
        fp+=(len(pred_toks)-num_same)
        tn+=(len(gold_toks)-num_same)
    return tp,fp,tn

In [ ]:
with open("test_demo.pkl") as f:
    test_data=pickle.load(f)

In [ ]:
"""Converts a mini batch of training dataset, applies required padding and returns a feed dict"""
def train(x):
    feed_dict={}
    q_w = []
    q_c = []
    c_w = []
    c_c = []
    a_s = []
    a_e = []
    q_b = []
    c_b = []
    for i in x:
        zero_w = np.zeros((q_words-len(i['questionword']),word_embedding_dim))
        q_w.append(np.concatenate((i['questionword'],zero_w),axis=0))
        zero_w = np.zeros((c_words-len(i['contextword']),word_embedding_dim))
        c_w.append(np.concatenate((i['contextword'],zero_w),axis=0)) 
        q_b.append(np.pad(i['questionbool'],[0,q_words-len(i['questionword'])],'constant', constant_values=(2)))
        c_b.append(np.pad(i['contextbool'],[0,c_words-len(i['contextword'])],'constant', constant_values=(2)))
        zero_c = (no_of_chars-1)*np.ones((q_words-len(i['questionchar']),max_word_len))
        q_c.append(np.concatenate((i['questionchar'],zero_c),axis=0))
        zero_c = (no_of_chars-1)*np.ones((c_words-len(i['contextchar']),max_word_len))
        c_c.append(np.concatenate((i['contextchar'],zero_c),axis=0))
        a_s.append(i['answer_start'])
        a_e.append(i['answer_end'])
    feed_dict = {
        question_word:np.array(q_w),
        context_word:np.array(c_w),
        question_char:np.array(q_c),
        context_char:np.array(c_c),
        start_ans:np.array(a_s),
        end_ans:np.array(a_e),
        question_bool:np.array(q_b),
        context_bool:np.array(c_b)
    }

    return feed_dict

In [ ]:
"""Evaluate results"""
def test(x_test,max_f1,testing=True):
        tp=0
        fp=0
        tn=0
        no_of_test_batches=len(x_test)/batch_size
        t_ptr=0
        for tbatch in range(no_of_test_batches):
            test_feed = train(x_test[t_ptr:t_ptr+batch_size])
            t_ptr+=batch_size
            np_ans_st = sess.run(logits1_f,test_feed)
            np_ans_e = sess.run(logits2_f,test_feed)
            ans_st = np.argmax(np_ans_st,axis=-1)
            ans_e = np.argmax(np_ans_e,axis=-1)
            ans_pred=[range(x,y) for x,y in zip(ans_st,ans_e)]
            ans_ac=[range(x,y) for x,y in zip(test_feed[start_ans],test_feed[end_ans])]
            x,y,z=compute_f1(ans_pred, ans_ac)
            tp+=x
            fp+=y
            tn+=z
#             if tbatch==0:
#                 print np_ans_st[0]
            if tbatch%10==0:
                print("Testing %d" % tbatch),
                if testing:
                    print "On testing data"
                else:
                    print "On training data"
                print "Start: ",np.argmax(np_ans_st,axis=-1)[0],
                print "End: ",np.argmax(np_ans_e,axis=-1)[0]
                print "Start Actual: ",test_feed[start_ans][0],
                print "End Actual: ",test_feed[end_ans][0]
#             if tbatch==50:
#                 print "Global Step: ",sess.run(global_step)
#                 break
        print tp,fp,tn
        try:
            precision = 1.0 * tp / (tp+fp)
            recall = 1.0 * tp / (tp+tn)
            f1 = (2 * precision * recall) / (precision + recall)
            print("F1 Score:")+str(f1)
        except:
            print("F1 Score:N/A")
            if testing:
                save_path = saver.save(sess, "./saved_model2/model.ckpt")
                print("Model saved in path: %s" % save_path)
            return max_f1
        if f1>max_f1 and testing:
            save_path = saver.save(sess, "./saved_model2/model.ckpt")
            print("Model saved in path: %s" % save_path)
            max_f1 = max(f1,max_f1)
        else:
            print "Not Saved"
        return max(max_f1,f1)

In [ ]:
f1 = test(test_data,0,False)

In [ ]:
def testing_func(x):
    feed_dict={}
    q_w = []
    q_c = []
    c_w = []
    c_c = []
    a_s = []
    a_e = []
    q_b = []
    c_b = []
    for i in x:
        zero_w = np.zeros((q_words-len(i['questionword']),word_embedding_dim))
        q_w.append(np.concatenate((i['questionword'],zero_w),axis=0))
        zero_w = np.zeros((c_words-len(i['contextword']),word_embedding_dim))
        c_w.append(np.concatenate((i['contextword'],zero_w),axis=0)) 
        q_b.append(np.pad(i['questionbool'],[0,q_words-len(i['questionword'])],'constant', constant_values=(2)))
        c_b.append(np.pad(i['contextbool'],[0,c_words-len(i['contextword'])],'constant', constant_values=(2)))
        zero_c = (no_of_chars-1)*np.ones((q_words-len(i['questionchar']),max_word_len))
        q_c.append(np.concatenate((i['questionchar'],zero_c),axis=0))
        zero_c = (no_of_chars-1)*np.ones((c_words-len(i['contextchar']),max_word_len))
        c_c.append(np.concatenate((i['contextchar'],zero_c),axis=0))
    feed_dict = {
        question_word:np.array(q_w),
        context_word:np.array(c_w),
        question_char:np.array(q_c),
        context_char:np.array(c_c),
        question_bool:np.array(q_b),
        context_bool:np.array(c_b)
    }

    return feed_dict

In [ ]:
"""Returns the answer to the question given context"""
def get_answer(context,question):
    c_word,c_char,c_bool,c=preprocess(context)
    q_word,q_char,q_bool,q=preprocess(question)
    data = {'contextword':c_word,'contextchar':c_char,'contextbool':c_bool,'questionword':q_word,'questionchar':q_char,'questionbool':q_bool}
    example_feed = [data for _ in range(batch_size)]
    temp1 = np.argmax(sess.run(logits1,testing_func(example_feed))[0])
    temp2 = np.argmax(sess.run(logits2,testing_func(example_feed))[0])
    try:
        return context[c[temp1]:c[temp2]]
    except:
        return context[c[temp1]:]

In [ ]:
context=u"""Michael John Clarke (born 2 April 1981) is a former Australian international cricketer. He led Australia to their 5th Cricket World Cup triumph, when his team were victorious in the final of the ICC Cricket World Cup 2015 at the MCG. He is the first cap for Australia in Twenty20 Internationals. His ODI shirt number of 23 was passed on to him by Shane Warne after his international retirement. Nicknamed "Pup",[3] he is a right-handed middle-order batsman, an occasional left-arm orthodox spin bowler and also a slip catcher. He represented New South Wales at a domestic level. In January 2011, Clarke stood down as captain of the Australian Twenty20 cricket team to concentrate on his Test and ODI performance. After announcing he would retire from One Day cricket after the end of the 2015 Cricket World Cup, Clarke starred in the final against New Zealand top scoring with a score of 74 off 72 balls, as Australia won their fifth World Cup title. He was bowled when nine runs were required to win and received a standing ovation from the 93,013 strong MCG crowd after his dismissal. On 8 August 2015, Clarke announced that he would retire from all forms of cricket after the final Test of the 2015 Ashes series following a difficult series in terms of both his and the team's performance. Australia suffered a crushing defeat of an innings and 78 runs thus losing the Ashes. This was Clarke's fourth successive Ashes loss in England overall and his second as captain."""

In [ ]:
question=u"""When was Michael Clarke born?"""

In [ ]:
print(get_answer(context,question))